# FIRE 

This notebook will try to answer the following questions:

- Given my current savings, the monthly amount income, and the annual return on investment, how much money I could end up with in the next X years
- Given my current savings and no income for how many months will my savings last


## Assumptions

- The annual return on the stock investment (assumption ETF) is an average and is applied monthly. 
The assumption is a long term.
- The bond investment has a fixed return rate that's applied monthly. In reality that should be close to inflation rate.
- The monthly income is constant and is added to cash every month. It's also a simplification as in reality the income is not constant
- The monthly expenses are constant but the inflation rate is applied. In reality, the expenses won't be constant as they change depending on many factors.
- The inflation rate is constant


In [9]:
import sys
from pathlib import Path

project_root = Path.cwd().parent 
src_path = project_root / 'src'

sys.path.append(str(src_path))

In [10]:
import datetime
from fire.simulations import FireSimulation, simulate_next
import pandas as pd
import plotly.express as px


In [11]:
from fire.simulations import run_simulation


init = FireSimulation(
    stock_investments=100_000,
    bonds_investments=100_000,
    properties_market_value=0,
    properties_monthly_income=0,
    cash=50_000,
    return_rate_from_investment=0.06,
    return_rate_from_bonds=0.04,
    monthly_expenses=20_000,
    annual_inflation_rate=0.04,
    monthly_income=0,
    annual_property_appreciation_rate=0.03,
    invest_cash_surplus=True,
    invest_cash_threshold=50_000,
    invest_cash_surplus_strategy='80-20',
    start_date=datetime.datetime.fromisoformat('2024-03-01')
)

# simulate for next X years
years = 10
simulation = run_simulation(init, years*12)

In [12]:

df = pd.DataFrame([s.to_dict() for s in simulation])
fig = px.line(df, x='start_date', y=['monthly_expenses' ,'stock_investments', 'bonds_investments', 'properties_market_value', 'cash', 'wealth'])

fig